In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imp
from camera import camera3d as cam
import calibrator as cal
import cv2
import imp
import time
import numpy as np
import os
import subprocess
import skg
from skspatial.objects import Plane
from skspatial.objects import Points
import imageio

pathFile = "./images"

# 1 - Define simple function to create and navigate folders

In [ ]:
def createPath(path,params = []):
    if not os.path.exists(path):
        os.makedirs(path)
    for param in params:
        path = path + '/' + str(param)
        if not os.path.exists(path):
            os.makedirs(path)
    return path
    
    
def checkPath(path,params = []):
    for param in params:
        path = path + '/' + str(param)
    return path

# 2 - Get Bowl

no IR Filters

no IR Lights

no Projection

In [ ]:
nxBowl = 5

frames = []
position = []
camera = cam.Camera(rgbMode=0,depthMode = 0,infraredMode = 0,infrared = True,filtre = False)
camera.alignTo(True)
camera.setExposureIR(20000)
camera.setGainIR(16)

In [ ]:
w = np.arange(0,1280,1)
h = np.arange(0,720,1)
tmp = camera.getFramesAligned()
path = createPath(pathFile,["bowl3D","IR"])

xBowl = []
for k in range(0,5):
    tmp = camera.getFramesAligned()
    X = np.zeros((720,1280,3))
    for x in w:
        for y in h:
            point = camera.getWorldPosition(x,y)
            
            X[y,x,:] += point
    xBowl.append(X)

In [ ]:
camera.stop()

# 3 - Turn on the projection

In [ ]:
frames = []
position = []
controller = cal.Controller()
camera = cam.Camera(infraredMode=0,infrared = True,rgb = False,depth=True,filtre = False)
camera.alignTo(True)
camera.laserSwitch()
exposure = 10000
camera.setExposureIR(exposure)

# 4 - Get mask

In [ ]:
def createMask(frame,rate):
    ret,thres = cv2.threshold(frame,100,255,cv2.THRESH_BINARY)
    contours,hierarchy = cv2.findContours(thres ,cv2.RETR_LIST ,cv2.CHAIN_APPROX_SIMPLE)
    maskCircle = [0,0,0]
    for contour in contours:
        (x,y),radius = cv2.minEnclosingCircle(contour)
        center = (int(x),int(y))
        radius = int(radius * rate)
        #cv2.circle(frame,center,radius,(255,0,0),2)
        if maskCircle[0]<radius:
            maskCircle[0] = int(radius * rate)
            maskCircle[1:] = center
    mask = frame * 0
    cv2.circle(mask,maskCircle[1:],maskCircle[0], 1,-1, 8,0)

    return mask


In [ ]:
controller.calibrationColor(1,0,1,True)
controller.calibrationPosition(10,10)
controller.calibrationScale3(1,1,1)
for k in range(0,10):
    X = camera.getFramesAligned()

X = camera.getFramesAligned()

maskIR = createMask(X[1],1)
maskIR9 = createMask(X[1],.9)

# 5 - Get background

In [ ]:
def backGroundSubstractor(frames):
    fgbg = cv2.createBackgroundSubtractorMOG2() 
    for frame in frames:
        fgmask = fgbg.apply(frame,learningRate=0.01)
    return fgbg

def backgroundSubstract(frame,fgbg):
    fgmask = fgbg.apply(frame,learningRate=0)
    return fgmask

In [ ]:
path = createPath(pathFile,["lines","10000","bg"])
controller.calibrationColor(1,0,0,True)
for k in range(0,10):
    X = camera.getFramesAligned()
irImages = []
for k in range(0,20):
    X = camera.getFramesAligned()
    irImages.append(X[1])
fgbg = backGroundSubstractor(irImages)


In [ ]:
camera.stop()
controller.stopDisplay()
controller.disconnect()

# 6 - Fit bowl

In [ ]:
def flattener(arr):
    return arr.reshape(-1,arr.shape[-1])

def stackAll(x,nx = 5):
    xA = flattener(x[0])
    for k in range(1,nx):
        xA = np.vstack((xA,flattener(x[k])))
    xA = xA[~np.isnan(xA).any(axis=1), :]
    xA = xA[~(xA == 0).any(axis=1), :]
    return xA

In [ ]:
def halfSphereSample(nx = 1000):
    theta = np.linspace(0,np.pi/2.0,nx)

    points = np.zeros((0,3))
    for k in range(0,nx):
        nPhi = int(1 + nx*np.cos(theta[k]))

        dPhi = 2*np.pi/nPhi
        phi = np.linspace(-np.pi,np.pi-dPhi,nPhi)
        
        X = np.zeros((nPhi,3))
        
        X[:,0] = np.cos(theta[k])*np.cos(phi)
        X[:,1] = np.cos(theta[k])*np.sin(phi)
        X[:,2] = np.sin(theta[k])

        points = np.vstack((points,X))
    return points

In [ ]:
imp.reload(cam)
maskIR3D = np.moveaxis(np.tile(maskIR,(3,1,1)),0,2)
for k in range(0,5):
    xBowl[k] *= maskIR3D
xAll = stackAll(xBowl,5)
sphere = skg.nsphere_fit(xAll)
sphereBowl = sphere[0]*halfSphereSample(nx = 2000)+sphere[1]

camera = cam.Camera(rgbMode=0,depthMode = 0,infraredMode = 0,infrared = True,filtre = False)
camera.laserSwitch()
camera.alignTo(True)

bowlFitted = camera.simulateDepth(sphereBowl)
bowlFittedNormalized = ((bowlFitted - sphere[1])/sphere[0])
bowlFittedNormalized2 = np.zeros((720,1280,2))
bowlFittedNormalized2[:,:,0] = np.arctan2(bowlFittedNormalized[:,:,1],bowlFittedNormalized[:,:,0])
r = np.sqrt(bowlFittedNormalized[:,:,1]**2+bowlFittedNormalized[:,:,0]**2)
bowlFittedNormalized2[:,:,1] = np.arcsin(bowlFittedNormalized[:,:,2])

camera.stop()

# 6a - Write Down Sphere properties 

In [ ]:
print(sphere)

# 7 - Get scanlines

In [ ]:
frames = []
position = []
controller = startController()
camera = cam.Camera(infraredMode=0,infrared = True,rgb = False,depth=True,filtre = False)
camera.alignTo(True)
camera.laserSwitch()
exposure = 10000
camera.setExposureIR(exposure)

In [ ]:
controller.calibrationMesh(1)
controller.calibrationPosition(.5,0)
controller.calibrationScale3(0.1,10,.1)
controller.calibrationColor(1,0,1,False)
controller.calibrationScale3(2/1920,10,2/1920)
controller.calibrationPosition(k,0)

In [ ]:
widths = []
heights = []

x = np.linspace(-1,1,192)
controller.calibrationScale3(2/1920,10,2/1920)
wIdx = []
hIdx = []
for k in x:  
    for j in range(0,10):
        X = camera.getFramesAligned()
    widths.append(X[1])
    wIdx.append(controller.calibrationPosition(k,0))

x = np.linspace(-1,1,108)

controller.calibrationScale3(10,2/1080,2/1080)
positionHeight = []
for k in x:
    for j in range(0,10):
        X = camera.getFramesAligned()
    heights.append(X[1])
    hIdx.append(controller.calibrationPosition(0,k))
camera.stop()
controller.stopDisplay()
controller.disconnect()

# 8 - Get Projected Grid

In [ ]:
def createBlobDetector(minArea = 200):
    params = cv2.SimpleBlobDetector_Params()

    params.filterByColor = False
    params.filterByArea = True
    params.minArea = minArea
    params.minThreshold = 10
    params.maxThreshold = 220
    params.filterByCircularity = False
    params.minCircularity = 0.1
    params.filterByConvexity = False
    params.minConvexity = 0.87
    params.filterByInertia = False
    params.minInertiaRatio = 0.01
    detector = cv2.SimpleBlobDetector_create(params)
    return detector

In [ ]:
detector = createBlobDetector(0)
coordX2 = np.zeros((len(hIdx),len(wIdx),3))*np.nan
coordPixel = np.zeros((len(hIdx),len(wIdx),2))*np.nan
for k in range(0,len(widths)):
    for j in range(0,len(heights)):
        try:
            point = detector.detect(100*widths[k]*heights[j])[0].pt
            coordX2[j,k,:] = bowlFittedNormalized[int(point[1]),int(point[0]),:]
            coordPixel[j,k,:] = [wIdx[k]-540.5,hIdx[j]-960.5]
        except Exception as e:
            pass


# 9 - Mapping Fit

In [ ]:
def rotationMatrix(x1,x2):
    a = np.cross(x1,x2)
    na = np.linalg.norm(a)
    a /= na
    u = np.linalg.norm(x1)
    v = np.linalg.norm(x2)
    d = np.dot(x1,x2)
    cosP = d / (u*v)
    sinP = na /(u*v)
    rot = [[cosP+(1-cosP)*a[0]**2,(1-cosP)*a[0]*a[1]-sinP*a[2],(1-cosP)*a[0]*a[2]+sinP*a[1]],
    [(1-cosP)*a[0]*a[1]+sinP*a[2],cosP+(1-cosP)*a[1]**2,(1-cosP)*a[1]*a[2]-sinP*a[0]],
    [(1-cosP)*a[0]*a[2]-sinP*a[1],(1-cosP)*a[1]*a[2]+cosP*a[0],cosP+(1-cosP)*a[2]**2]]
    return np.array(rot)

In [ ]:
def computeMapping(coordX2,mapCenter,mapStep = 30,projectorPosition = (0,0,3),phi0 = -0.001,theta0 = +0.01):

    rx = coordX2[mapCenter[0],mapCenter[1]-3:mapCenter[1]+3,:]
    rx = rx[~np.isnan(rx[:,0]),:]
    rotX = - np.arctan2(stats.linregress(rx[:,0],rx[:,1])[0],1)

    dx = np.linalg.norm((coordX2[mapCenter[0],mapCenter[1],:]-coordX2[mapCenter[0]-1,mapCenter[1],:])/2)
    dy = np.linalg.norm((coordX2[mapCenter[0],mapCenter[1],:]-coordX2[mapCenter[0],mapCenter[1]-1,:])/2)
    diff = 2*np.mean((dx,dy))/(mapStep)
    dPhi = np.arctan2(diff,np.linalg.norm(projectorPosition-coordX2[mapCenter[0],mapCenter[1],:]))

    return rotX,dx,dy,diff,dPhi,phi0,theta0,projectorPosition

In [ ]:
res = [1920,1080]
scale = 1
rotX,dx,dy,diff,dPhi,phi0,theta0,projectorPosition = computeMapping(coordX2,mapCenter)

h = theta0+(np.linspace(0,res[0],res[0]*scale)-959.5-mapCenterFull[1])*dPhi
w = phi0+(np.linspace(0,res[1],res[1]*scale)-539.5-mapCenterFull[0])*dPhi
phi =  np.tile(h,(res[1]*scale,1))
theta =  np.tile(w,(res[0]*scale,1)).T

w = np.zeros((res[1]*scale,res[0]*scale,3))*np.nan

w[:,:,0] = -np.sin(phi)*np.cos(theta)
w[:,:,1] = np.sin(theta)
w[:,:,2] = np.cos(phi)*np.cos(theta)

dotProduct = np.sum(w * projectorPosition,axis = 2)

d = dotProduct - np.sqrt(dotProduct**2-(np.linalg.norm(projectorPosition)**2-1))
d = np.moveaxis(np.tile(d,(3,1,1)),0,2)

xS =  projectorPosition - d *w
xSrotated = np.copy(xS)
xSrotated[:,:,0] = np.cos(rotX) *  xS[:,:,0] + np.sin(rotX) *  xS[:,:,1]
xSrotated[:,:,1] = - np.sin(rotX) *  xS[:,:,0] + np.cos(rotX) *  xS[:,:,1] 
xS = xSrotated

rot = rotationMatrix(xS[int(539.5+mapCenterFull[0]*scale),int(959.5+mapCenterFull[1]*scale),:],
                        coordX2[mapCenter[0],mapCenter[1],:])
xSrotated = applyRotationMatrix(rot,xS)

normalTexture =  xSrotated

# 10 - Write Texture

In [ ]:
Y = np.copy(normalTexture).astype("float32")
Y[:,:,1] = -normalTexture[:,:,2]
Y[:,:,2] = normalTexture[:,:,1]
imageio.imwrite("bowl.exr", (Y+1)/2.0)

# 11 - Get Water Surface

place the IR Filters

place a floating cardboard square on the surface of water

In [ ]:
frames = []
position = []

camera = cam.Camera(infraredMode=0,infrared = True,rgb = False,depth=True,filtre = False)
camera.alignTo(True)
camera.setHighAccuracy()
camera.setExposureIR(10000)
normals = []
centers = []
for k in range(0,10):
    for j in range(0,10):
        X = camera.getFramesAligned()
    #obviously this 300-500 range is kind of shit and should be automated
    ret,thres1 = cv2.threshold(X[0],300,1,cv2.THRESH_BINARY)
    ret,thres2 = cv2.threshold(X[0],500,1,cv2.THRESH_BINARY_INV)
    #plt.matshow(thres1*thres2*maskIR9)
    idx = np.where(thres1*thres2*maskIR9)
    surfaceX = []
    for l in range(0,len(idx[0])):
        point = camera.getWorldPosition(idx[0][l],idx[1][l])
        surfaceX.append(point)
points = Points(surfaceX)
plane = Plane.best_fit(points)   
normal = np.mean(normals,axis = 0)
center = np.mean(centers,axis = 0)
rotWater = rotationMatrix(plane.normal,[0,0,1])
rotWaterHeight = plane.point

# 11a - Write Down Water Surface properties 

In [ ]:
print(plane)